# Introducción a Topic Modeling

Topic modeling es una técnica de aprendizaje automático no supervisado donde intentados descubrir tópicos que son abstractos al texto pero que pueden describir una colección de documentos. Es importante marcar que estos "tópicos" no son necesariamente equivalentes a la interpretación coloquial de tópicos, sino que responden a un patrón que emerge de las palabras que están en los documentos.

La suposición básica para Topic Modeling es que cada documento está representado por una mescla de tópicos, y cada tópico consite en una conlección de palabras.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from tqdm import tqdm

In [ ]:
tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [ ]:
tweets.groupby('SECTOR').head(1)[['TEXTO', 'SECTOR']]

# Preprosesamiento

## Stop words

Algunas palabras que son extremadamente frecuentes, "a-priori" (revisaremos este concepto luego) no son de mucha utilidad para resolver una tarea de clasificación de texto específica. Estas palabras se las conoce como Stop words y, dado que son de poca utilidad, son eliminadas del texto.

**Spoiler Alert:**
Mencionamos 'a priori', porque la tendencia general en los ultimos tiempos ha sido ir desde grandes listas de stop words en el order de 200-300 a listas muy pequeñas (10-15 - si es que las hay). Los buscadores, por ejemplo, hoy en día no eliminan estas palabras. Cuando veamos modelos de lenguaje, en realidad las vamos a necesitar.



In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
spa_stopwords = stopwords.words('spanish')

In [ ]:
spa_stopwords[:10]

## Tokenización

Se refiere al proceso de generación de tokens basado en un texto. Un token se diferencia de una palabra en el hecho de que una palabra es una instancia de un token. Existen varias técnicas para separar una oración o texto en general en palabras discretas.

Lectura recomendada: Diferentes tokenizers:
 - http://www.nltk.org/api/nltk.tokenize.html

In [ ]:
tweet = tweets['TEXTO'][5]
print(tweet)

In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

In [ ]:
from nltk.tokenize.casual import TweetTokenizer

tokenizer = TweetTokenizer()

## Stemming and Lemmatization

Existen palabras cuyo significado no cambia ya que estan atados a una palabra raiz que les da el significado:

<i>Organizan, organso, organiza, organizando</i>

**Stemming y Lemmatization** son dos técnicas que generan la palabra raiz dada una palabra. La diferencia que hay entre estas técnicas es que **Lemmatization** utiliza reglas del lenguaje para extraer las palabras raiz y por lo tanto, el resultado son palabras que existen en el vocabulario. Por el contrario, **Stemming** utiliza heuristicas que truncan la palabra hasta su raiz invariable. El resultado son "psudopalabras" o mejor conocidos como tokens que no forman una palabra del lenguaje propiamente dicho. Esta técnica, como se puede intuir, es más rápida computacionalmente. 

In [ ]:
from nltk import stem

import spacy
from spacy.lemmatizer import Lemmatizer

**Sobre la libreria spaCy:** Spacy es una libreria para NLP muy polupar actualmente ya que, al contrario de nltk, ofrece formas muy eficientes de hacer solo algunos tipos de operaciones. NLTK es una herramienta más general. Para instalar spaCy en español necesitaran ejecutar:

```
conda install -c spacy spacy
python -m spacy download es_core_news_sm
```

Si bien NLTK ofrece la opción de hacer Lemmatization, su soporte mayoritariamente es para ingles. La versión en español no es demasiado buena. Si les interesa probarla puede hacerlo a traves del metodo.

```
nltk.wordnet.lemas("palabra", lang='spa')
```

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
parser = spacy.load('es_core_news_sm')

In [ ]:
lemmatizer = lambda word : " ".join([token.lemma_ for token in parser(word)])
stemmer = stem.SnowballStemmer(language='spanish')

In [ ]:
words = ['amigos', 'amigo', 'amiga', 'amistad' ]

In [ ]:
[stemmer.stem(word) for word in words]

In [ ]:
[lemmatizer(word) for word in words]

Nota: La precisión de Lemmatization depende de la implementación. La de español no es demasiado buena. Algunas palabras podrian no encontrarse.

### Otra estrategia

El problema de reducir las palabras a sus formatos raiz radica en que en general cada palabra (separada por espacios, puntos, etc) conforma un elemento en nuestro vocabulario y no queremos diferentes elementos de nuestro vocabulario que mapeen al mismo elemento o concepto. Si por el contrario utilizaramos otra estrategia para determinar nuestro vocabulario (o mejor dicho, cada elemento de nuestro vocabulario) entonces este problema quizás no existiría (o se volvería peor).

Este tipo de técnicas por lo general intentan representar el vocabulario con "sub-palabras" o partes de las palabras como unidad. Un ejemplo de esto es SentencePiece.

## Creando una rutina de preprosesamiento de texto

Adicionalmente de utilizar Lemmatization y eliminar stop words, necesitamos hacer algunas tareas extras:
 - Eliminar caracteres especiales: Acentos y caracteres especiales podrían complejizar el la representación de palabras, por lo que los eliminaremos.
 - Eliminaremos URLs y handles que son típicos en tweeter. Esto es especifico en este set de datos ya que una URL no representa información en este contexto.

In [ ]:
import unidecode
import spacy
from nltk import stem
from nltk.corpus import stopwords
from nltk.tokenize.casual import TweetTokenizer

parser = spacy.load('es_core_news_sm')
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
stemmer = stem.SnowballStemmer(language='spanish')
lemmatizer = lambda word : " ".join([token.lemma_ for token in parser(word)])
stopwords = set(stopwords.words('spanish'))

def process_text(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [unidecode.unidecode(token) for token in tokens]
    tokens = [lemmatizer(token) for token in tokens]
    return tokens

In [ ]:
doc_list = []

for doc in tqdm(tweets['TEXTO']):
    tokens = process_text(doc)
    doc_list.append(tokens)

Revisemos algunos resultados:

In [ ]:
tweets['TEXTO'][5]

In [ ]:
doc_list[5]

## Vectorización

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
vectors = vectorizer.fit_transform(doc_list).todense()

In [ ]:
vectors.shape

¿Que representa 7665?

In [ ]:
vocab = np.array(vectorizer.get_feature_names())

In [ ]:
vocab[4040:4050]

## Métodos básados en SVD

Los modelos basados en factorización de matrices intentan reducir la dimensionalidad de la matriz al aproximarla usando dos matrices más pequeñas con <i>k</i> factores latentes. Este método es bastante popular no solo en NLP sino que también en sistemas de recomendación, método que fué ganador del Netflix Prize (Funk SVD).

El concepto de decomposición de matrices es muy similar al de PCA en el sentido de que el número de factores latentes determina la cantidad de concepto abstractos que queremos mapear en un espacio dimensional menor. A medida que agregamos factores latentes, aumentaremos la especificación de los mismos hasta que llegue un momento donde los factores serán demasiados y el modelo comience a saturarse (over-fitting).

<img src="https://miro.medium.com/max/700/1*Z0EUVs7QElEqRqXtqut_FQ.png" />


U y V(trapuesta) son ortogonales. Esto es de esperar porrque si determinadas propiedades determinan un determinado factor latente, entonces esas propiedades serán poco relevantes en los restantes factores (pues sino, no haría sentido que conformen un factor distinto en un primer lugar).

SVC es un metodo de decomposición exacto, lo que singnifica que las matrices U y V son lo suficientemente grandes para mapear exactamente la matriz A.

## LSI - Latent Semantic Indexing

Cuando SVD es utilizado para procesar tópicos en texto y en donde los valores de la matriz A corresponden a frecuencias de palabras (ya sea por su frecuencia de aparición o con el método TF-IDF), este método se lo denomina Latent Semantic Analysis (sin embargo, en NLP no se lo suele nombrar como LSI).

<img src='https://github.com/fastai/course-nlp/raw/aabfeddf61fea29b18c72f841d057b56a216b7eb/images/svd_fb.png' />

Facebook Research: Fast Randomized SVD [https://research.fb.com/fast-randomized-svd/])

En esta configuración entonces:
 - A es una matriz de m x n donde m es la cantidad de documentos ú observaciones, y n es la cantidad de palabras en el vocabulario.
 - Los valores de A corresponden a la frecuencia de la cada palabra del vocabulario en cada observación ú documento.
 
Adicionalmente, dado que SVC es un método de decomposición exacto, tiende a producir matrices de poca densidad (sparse). Para evitar este problema, se utiliza una versión modificada de SVC conocida como Truncated SVD que solamente computa los k componentes mas grandes en la descomposición. Esto ayuda a que LSI combata efectivamente el problema de matrices sparse que tienden a generarse cuando se tienen cuerpos de texto con sinónimos y palabras que significan varias cosas dependiendo del contexto. Truncated SVD evíta ser un método de decomposición exacto al trabjar con una matriz Q que satisface:

$$A \approx QQ^*A $$

Métodos para generar Q pueden ser encontrados en el paper: Finding structure with randomness: Probabilistic algorithms for constructing approximate matrix decompositions [https://arxiv.org/abs/0909.4061]

In [ ]:
from sklearn.decomposition import TruncatedSVD

vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')
vectors = vectorizer.fit_transform(doc_list).todense()

TF-IDF es una forma de normalizar los vectores de frecuencias al tomar en consideración la frecuencia en la que aparece la palabra en el documento, la longitud del documento y que tan comun o raro es la palabra en todo el corpus.

$$TF = \frac {freq(w_i)} {len(doc)} $$


$$IDF = log(\frac {len(corpus)} {freq(w_i, corpus)}) $$

In [ ]:
svd = TruncatedSVD(n_components=7, algorithm='randomized')
USigma = svd.fit_transform(vectors)
Sigma = svd.singular_values_
VT = svd.components_

In [ ]:
VT.shape

Internamente, TrucatedSVC es un wrapper de la clase randomized_svd donde la matríz Q que vimos anteriormente se genera a través de un método de sampling aleatorio. Las siguientes lineas son equivalentes a lo que vimos anteriormente:

In [ ]:
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(vectors, 
                              n_components=7,
                              n_iter=5)

Podemos validar que U es una matriz ortogonal

In [ ]:
np.allclose(U.T @ U, np.eye(U.shape[1]))

Si vemos los valores de la matriz Sigma, veremos la importancia relativa de los documentos con respecto a los tópicos que encontramos. Si los gráficamos vemos que sus valores comienzan a decrecer relativamente rápido, sosteniendo la supoción de que Truncated SVD genera los K más relevantes tópicos.

In [ ]:
plt.plot(Sigma)

### Interpretando los tópicos


In [ ]:
def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[-8:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [ ]:
show_topics(VT)

Limitaciones en SVD:
 - SVD sufre de un problema llamado "Indeterminación del signo", que básicamente significa que el signo en la matríz VT y USigma dependen del algorimo que se utilizó para generarlos y de las condiciones iniciales (initial random state). En este contexto, que significa que un tópico esté relacionado con una palabra en un valor negativo?

## NMF: Non-negative Matrix Factorization

Motivación: En lugar de construir nuestros factores imponiendo la restricción de que sean ortogonales, la idea es de construirlos de tal forma que sean no-negativos.

In [ ]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=7, random_state = 1234)

In [ ]:
W1 = nmf.fit_transform(vectors)
H1 = nmf.components_

In [ ]:
H1.shape

In [ ]:
show_topics(H1)

## LDA: Latent Dirichlet Allocation

LDA es un método Bayesiano basado en la distribución de Dirichlet, la cual es una distribución sobre probabilidades en K categorias. LDA supone que los documentos que tenemos pertenecen a K categorias distintas cuya distribución es desconocida.

La distribución Dirichlet es una generalización de la distribución Beta en un espacio multidimensional. Así como la distribución beta es la distribución previa de la binomial, la distribución de Dirichlet es la distribución previa de la multinomial. 

$$ P(w\mid d) = P(d)\sum_c P(k\mid d)P(w\mid k) $$

David Blei, Andrew Ng, Michael Jordan:  Latent Dirichlet Allocation [https://jmlr.org/papers/volume3/blei03a/blei03a.pdf]

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=7)

In [ ]:
lda.fit(vectors)

In [ ]:
for idx, topic in enumerate(lda.components_):
    print ("Topic ", idx, " ".join(vocab[i] for i in topic.argsort()[:-10 - 1:-1]))

# Creando un pipeline de preprocesamiento de texto

A pesar de que los métodos anteriores son no supervisados, son de utilidad para el modelado de de problemas no supervisados como supervisados. Para llevar estos métodos a un entorno práctico normalmente se construyen flujos de procesamiento como el que se muestra más abajo:

<img src='Docs/atap_0406.png' />

A modo de ejemplo, el siguiente codigo utiliza la API de Scikit-Learn para generar el paso de normalización de texto. Este "paso" lo podemos insertar en un pipeline de Machine Learning que luego utilicemos para resolver una tarea en particular

In [ ]:
import unidecode
import spacy
import sklearn

from nltk import stem
from nltk.corpus import stopwords
from nltk.tokenize.casual import TweetTokenizer

class TextNormalizer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, language='spanish'):
        parser = spacy.load('es_core_news_sm')
        tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
        stemmer = stem.SnowballStemmer(language=language)
        lemmatizer = lambda word : " ".join([token.lemma_ for token in parser(word)])
        stopwords = set(stopwords.words(language))
    
    def process_text(text):
        tokens = tokenizer.tokenize(text)
        tokens = [token for token in tokens if len(token) > 4]
        tokens = [token for token in tokens if token not in stopwords]
        tokens = [unidecode.unidecode(token) for token in tokens]
        tokens = [lemmatizer(token) for token in tokens]
        return tokens
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for doc in X:
            yield ' '.join(self.process_text(doc))

Importamos algunas librerias que necesitaremos

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Instanciamos nuestro preprocesamiento de texto

In [ ]:
normalizer = TextNormalizer()

Instanciamos nuestro vectorizador, en este caso usando el método TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')

Instanciamos nuestro generador de features, que en este caso son los tópicos que LDA genere

In [ ]:
featurizer = LatentDirichletAllocation(n_components=7)

Instanciamos nuestro clasificador que utilizará las features generadas hasta este momento

In [ ]:
estimator = LogisticRegression(max_iter=10000, tol=0.1)

Creamos un pipeline que ejecute todos los pasos en secuencia

In [ ]:
pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', vectorizer),
                           ('featurizer', featurizer),
                           ('estimator', estimator)])

In [ ]:
pipeline.fit(tweets['TEXTO'], tweets['SECTOR'])

In [ ]:
predictions = pipeline.predict(tweets['TEXTO'])

In [ ]:
print(classification_report(tweets['SECTOR'], predictions))